In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("drive/My Drive/PA-GAN-Tensorflow-master")

In [3]:
!pip install tensorflow-graphics-gpu --no-deps
!pip install oyaml
#!pip install tensorflow-gpu==2.4.0

     |████████████████████████████████| 245kB 4.1MB/s 


In [4]:
CUDA_VISIBLE_DEVICES=0 
#!python test_multi.py --test_att_names Bangs Mustache 
!python test.py 

2021-06-10 15:42:48.169065: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-10 15:42:59.759712: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-10 15:42:59.848092: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-06-10 15:42:59.848165: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3e5887586f00): /proc/driver/nvidia/version does not exist
  0% 0/1 [00:00<?, ?it/s]2021-06-10 15:43:07.410548: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz
2021-06-10 15:43:07.489832: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
100% 1/1 [00:10<00:00, 10.46s/it]


In [6]:
import cv2
import numpy as np
import pylib as py
import tensorflow as tf
import tflib as tl
import tqdm
import data

def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

output_dir = 'output/PA-GAN_256/'
args = py.args_from_yaml(output_dir + 'settings.yml')
n_atts = len(args.att_names)

sess = tl.session()
sess.__enter__()  

# data
test_dataset, len_test_dataset = data.make_celeba_dataset(args.img_dir, args.test_label_path, args.att_names, args.n_samples,
                                                          load_size=args.load_size, crop_size=args.crop_size, repeat=None)

test_iter = tf.compat.v1.data.make_one_shot_iterator(test_dataset)


with tf.io.gfile.GFile((output_dir + 'generator.pb'), 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='generator')
xa = sess.graph.get_tensor_by_name('generator/xa:0')
a_ = sess.graph.get_tensor_by_name('generator/a_:0')
b_ = sess.graph.get_tensor_by_name('generator/b_:0')
x = sess.graph.get_tensor_by_name('generator/xb:0')
e = sess.graph.get_tensor_by_name('generator/e:0')
ms = sess.graph.get_operation_by_name('generator/ms').outputs
save_dir = './output/%s/out' % (args.experiment_name)
mkdir(save_dir)
cnt = 0
for _ in tqdm.trange(1):
    # data for sampling
    xa_ipt, a_ipt = sess.run(test_iter.get_next())
    b_ipt_list = []  
    att = [1,2,3,4,6,9,10,12]
    for i in att:
        tmp = np.array(a_ipt, copy=True)
        tmp[:, i] = 1 - tmp[:, i]   
        tmp = data.check_attribute_conflict(tmp, args.att_names[i], args.att_names)
        b_ipt_list.append(tmp)
    x_opt_list = []
    e_opt_list = [np.full_like(xa_ipt, -1.0)]
    ms_opt_list = []
    a__ipt = a_ipt * 2 - 1
    for i, b_ipt in enumerate(b_ipt_list):
        b__ipt = (b_ipt * 2 - 1).astype(np.float32) 
        if i > 0:  
            b__ipt[..., i - 1] = b__ipt[..., i - 1] * args.test_int
        x_opt, e_opt, ms_opt = sess.run([x, e, ms], feed_dict={xa: xa_ipt, a_: a__ipt, b_: b__ipt})       
        cv2.imwrite('%s/%d.jpg' % (save_dir, i),cv2.cvtColor((127.5*x_opt[0]+127.5).astype('uint8'), cv2.COLOR_BGR2RGB ))

sess.close()


100%|██████████| 1/1 [00:03<00:00,  3.48s/it]
